In [1]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model, Dataset
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np



In [2]:
ds = Dataset.from_local_or_download(
    'yack'
)
print('structure', len(ds.structure.true))
print('dms', len(ds.dms.true))
print('shape', len(ds.shape.true))

yack: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

yack: Download complete. File saved at data/yack/data.json
Loading dataset from HF
Done!                            
structure 133456
dms 133456
shape 133456


In [40]:
dm = DataModule(
    name=['archiveII', 'RNAStralign','bpRNA'],
    batch_size=32,
    train_split=None,
)
dm.setup('fit')

Loading dataset from disk
Done!                            


In [41]:
for batch in dm.train_dataloader():
    print(batch.get('dms'))
    break

tensor([[-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        ...,
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.]],
       device='mps:0')


In [42]:
import json

name = 'yack'
data = json.load(open(f'./data/{name}/data.json'))

In [46]:
from dmsensei.core.util import _pad, base_pairs_to_pairing_matrix
import tqdm 

def compare_data(dt, data_line, from_ds, ref):
    from_ds = from_ds.cpu().reshape(-1).tolist()
    
    if not dt in data_line:
        assert from_ds is None or set(from_ds) == set([-1000.]), f'{dt} is None in the json and not not in the datamodule for {ref}: {from_ds}'
        return
    L = len(data_line['sequence'])
    data_line = data_line[dt]
    
    if dt == 'structure':
        data_line = base_pairs_to_pairing_matrix(data_line, L, padding=len(from_ds))

    else:
        from_ds = np.array(from_ds)[:L].round(4)
        data_line = np.array(data_line).round(4)
    
    if not (data_line == from_ds).all():
        assert 0, f'{dt} keys do not match for {ref}: \ndata_line:\t{data_line} \nfrom_ds:\t{from_ds}'
    return

def sanity_check(data:dict, dm:DataModule):
    for batch in tqdm.tqdm(dm.train_dataloader(), total=len(dm.train_dataloader())):
        for ref, dms, shape, structure in zip(batch.get('reference'), batch.get('dms'), batch.get('shape'), batch.get('structure')):
            for dt, from_ds in zip(['dms', 'shape', 'structure'], [dms, shape, structure]):
                if dt == 'structure':
                    continue
                compare_data(dt, data[ref], from_ds, ref)

sanity_check(data, dm)

 67%|██████▋   | 2796/4171 [01:14<00:35, 38.89it/s]